In [1]:
#%pip install pandas
#%pip install seaborn
#%pip install openpyxl
#%pip install scipy

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import friedmanchisquare

In [16]:
df = pd.read_excel("Consolidado_Final_PSNR.xlsx")
# eliminar columnas que no se usan Ejecucion
df = df.drop(columns=["Ejecucion"])

In [18]:
import pandas as pd
import scipy.stats as stats
import numpy as np

algorithm_columns = df.columns[3:]

# Lista para almacenar los resultados del test de Friedman
results_list = []

# Aplicar el test de Friedman para cada grupo de función objetivo, imagen y dimensión
for name, group in df.groupby(['Funcion_Objetivo', 'img', 'Dimension']):
    # Extraer los datos de todos los algoritmos para este grupo
    data = [group[alg].dropna().values for alg in algorithm_columns]  # Asegúrate de descartar NA
    
    # Solo realizar el test si hay datos suficientes y todos los algoritmos tienen datos
    if len(data) >= 2 and all(len(d) > 1 for d in data):
        # Aplicar el test de Friedman
        friedman_stat, p_value = stats.friedmanchisquare(*data)
        
        # Calcular rangos promedio para cada algoritmo
        ranks = stats.rankdata([-np.mean(d) for d in data], method='average')  # -np.mean para ordenar de mayor a menor
        
        # Crear un diccionario para los resultados y los rangos
        result_dict = {
            'Funcion_Objetivo': name[0],
            'Imagen': name[1],
            'Dimension': name[2],
            'Friedman_Stat': friedman_stat,
            'P_Value': p_value
        }
        
        # Agregar rangos al diccionario
        for i, alg in enumerate(algorithm_columns):
            result_dict[alg + '_Rank'] = ranks[i]
        
        # Añadir el diccionario a la lista de resultados
        results_list.append(result_dict)

# Crear DataFrame de resultados a partir de la lista de diccionarios
results = pd.DataFrame(results_list)

results.tail()


,Funcion_Objetivo,Imagen,Dimension,Friedman_Stat,P_Value,RSA_Rank,HBA_Rank,OPA_Rank,BES_Rank,GWO_Rank,CSA_Rank,HHO_Rank,TSO_Rank
115,otsu,19,8,46.155556,8.153321e-08,4.0,3.0,5.0,8.0,1.0,2.0,6.0,7.0
116,otsu,19,9,34.077778,1.665684e-05,5.0,3.0,1.0,7.0,2.0,4.0,6.0,8.0
117,otsu,20,7,76.188889,8.223209e-14,6.0,4.0,3.0,8.0,1.0,2.0,5.0,7.0
118,otsu,20,8,93.433333,2.435950e-17,6.0,4.0,2.0,8.0,1.0,3.0,5.0,7.0
119,otsu,20,9,93.244444,2.664008e-17,6.0,3.0,5.0,8.0,1.0,2.0,4.0,7.0


In [19]:
results.to_excel("Friedman_PSNR.xlsx")

In [117]:
# Calcular el rango promedio por función objetivo, imagen y dimensión para cada algoritmo
average_ranks_all = results.groupby(['Funcion_Objetivo', 'Imagen', 'Dimension'])[[alg + '_Rank' for alg in algorithm_columns]].mean()

# Determinar el mejor algoritmo por función objetivo, imagen y dimensión (el que tiene el rango promedio más bajo)
best_algorithm_all = average_ranks_all.idxmin(axis=1)

# Convertir el índice múltiple en columnas para mejor visualización
best_algorithm_all = best_algorithm_all.reset_index()

# Renombrar la columna 0 para reflejar que contiene el nombre del mejor algoritmo
best_algorithm_all.rename(columns={0: 'Mejor_Algoritmo'}, inplace=True)

# Mostrar los mejores algoritmos por función objetivo, imagen y dimensión
#print("Mejor Algoritmo por Función Objetivo, Imagen y Dimensión:")
#print(best_algorithm_all)


In [118]:
# guardar en excel
best_algorithm_all.to_excel("Mejor Algoritmo por Función Objetivo, Imagen y Dimensión.xlsx")